In [1]:
from glob import glob
import os
import fitz  # PyMuPDF 라이브러리
import pdfplumber
import sys
import re
import pandas as pd
from tqdm import tqdm
import traceback


c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def find_sentences(text, start_word, end_word='\n'):
    pattern = re.compile(f'{start_word}.*?\{end_word}', re.DOTALL)
    matches = pattern.findall(text)
    result = matches[0].replace(start_word, "")
    result = result.replace(end_word, "")
    return result

### use fitz
def read_pdf_use_fitz(file_path, page_number=0):
    doc = fitz.open(file_path)
    page = doc[page_number]
    texts = page.get_text()
    prod_name = find_sentences(texts, "상품명")
    prod_feat = find_sentences(texts, "상품특징", "거래조건")
    prod_name = prod_name.replace(":", "")
    prod_feat = prod_feat.replace(":", "")
    prod_feat = prod_feat.replace("\n", " ")
    prod_feat = prod_feat.replace("2.", "")
    doc.close()
    
    return prod_name.strip(), prod_feat.strip()
            
def pdf_to_dataframe(pdf_path):
    page_number = 0
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        if len(page.extract_tables()) == 0:
            table = page.extract_tables()[0]  # 여러 테이블 중 첫 번째 테이블 선택
        else:
            table = page.extract_tables()[-1]
        
        new_table = list()
        for i in table:
            new_table.append([None if j == "" else j for j in i])
        table = new_table
        df = pd.DataFrame(table)
        # 2개의 컬럼에서 모두 빈 공백인 행 삭제
        df = df.dropna(subset=df.columns, how='all')

        # 공백인 부분은 다음값으로 대체
        df.fillna(method='ffill', inplace=True)

        # 2, 3, 4, 5, 6 컬럼을 1번 컬럼과 함께 합치기
        df[1] = df[1].astype(str) + df.iloc[:, 2:].astype(str).agg(' '.join, axis=1)
        df[1] = df[1].str.replace("None", "")

        # 2, 3, 4, 5, 6 컬럼 삭제
        df = df.drop(df.columns[2:], axis=1)
    
        # 하나라도 null 값인 행 삭제
        df = df.replace("", pd.NA).dropna()

        # 첫 번째 행을 컬럼으로 사용
        df.columns = df.iloc[0]

        # 첫 번째 행은 더 이상 필요 없으므로 제거
        df = df[1:]
        columns = df.columns
        
        page_number += 1
        
        ### 그 다음 페이지의 표
        while True:
            try:
                page = pdf.pages[page_number]
                table = page.extract_tables()[0]  # 여러 테이블 중 첫 번째 테이블 선택
                etc = pd.DataFrame(table, columns=columns)
                
                ### 그 다음에도 불안전한 표인 부분 처리
                # 2개의 컬럼에서 모두 빈 공백인 행 삭제
                etc = etc.dropna(subset=columns, how='all')
                # 공백인 부분은 다음값으로 대체
                etc.fillna(method='ffill', inplace=True)
                # 하나라도 null 값인 행 삭제
                etc = etc.replace("", pd.NA).dropna()

                df = pd.concat([df, etc], axis=0)
                page_number += 1
            except Exception:
                # print("end page", page_number)
                page_number += 1
                pass
            finally:
                if page_number >= len(pdf.pages):
                    # print(f"End of the document. Processed {page_number} pages.")
                    break
    
    # 괄호 안의 텍스트와 괄호를 삭제하는 정규표현식
    df[columns[0]] = df[columns[0]].str.replace("\n", "")
    df[columns[0]] = df[columns[0]].str.replace(" ", "")
    df[columns[0]] = df[columns[0]].apply(lambda x: re.sub(r'\(.*', '', x))
    df.reset_index(drop=True, inplace=True)
    
    # null인 행들을 순회하면서 이전 행의 '내용'에 추가하고 해당 행 삭제
    null_rows = df[df[columns[0]]==""]
    for index, row in null_rows.iterrows():
        prev_index = index - 1
        if prev_index >= 0:
            try:
                df.at[prev_index, columns[1]] += f"\n{row[columns[1]]}"
            except KeyError:
                prev_index = prev_index -1
                df.at[prev_index, columns[1]] += f"\n{row[columns[1]]}"
        df.drop(index, inplace=True)
        
    df.reset_index(drop=True, inplace=True)
    
    
    
    # null인 행들을 순회하면서 이전 행의 '구분'에 추가하고 해당 행 삭제
    null_rows = df[df[columns[1]]==""]
    for index, row in null_rows.iterrows():
        prev_index = index - 1
        if prev_index >= 0:
            try:
                df.at[prev_index, columns[0]] += f"\n{row[columns[0]]}"
            except KeyError:
                prev_index = prev_index -1
                df.at[prev_index, columns[0]] += f"\n{row[columns[0]]}"
        df.drop(index, inplace=True)
        
    
    df[columns[0]] = df[columns[0]].str.replace("\n", " ")
    df.reset_index(drop=True, inplace=True)
    
    def change_spec(text):
        try:
            text = text.strip()
            if text.startswith("-"):
                # text = text[:1] + "●" + text[1 + 1:]
                text = text.replace("-", "●")
        except AttributeError:
            pass
        return text
    df[columns[1]] = df[columns[1]].apply(lambda x: change_spec(x))
    df.reset_index(drop=True, inplace=True)
    
    ## 구분에 중복되는 것은 병합해서 1개의 행으로 만듦
    df = df.groupby(df.columns[0])[df.columns[1]].agg(lambda x: '\n'.join(x)).reset_index()
    return df

In [223]:
# pdf_path = "../data/은행상품/신한은행/20060428003900000023LC000030.pdf"
# page_number = 0
# prod_name, prod_feat = read_pdf_use_fitz(pdf_path)
# # etc = pdf_to_dataframe(pdf_path)

In [31]:
###### 후처리
path = "../data/은행상품/신한_backup.csv"
df = pd.read_csv(path, encoding="utf-8-sig", index_col=0)

# df['기타사항'] = df['기타사항'].fillna('') + df['기타내용'].fillna('')
# df['기타사항'] = df['기타사항'].fillna('') + df['기타주요내용'].fillna('')
# df['기타사항'] = df['기타사항'].fillna('') + df['기타'].fillna('')
# df.drop(['기타내용', '기타주요내용', '기타'], axis=1, inplace=True)


# df['원금또는이자지급관련제한'] = df['원금또는이자지급관련제한'].fillna('') + df['원금및이자지급제한'].fillna('')
# df.drop(['원금및이자지급제한'], axis=1, inplace=True)

# df['판매기간및한도'] = df['판매기간및한도'].fillna('') + df['판매기간/한도'].fillna('')
# df['판매기간및한도'] = df['판매기간및한도'].fillna('') + df['판매기간'].fillna('')
# df.drop(['판매기간/한도', '판매기간'], axis=1, inplace=True)

# df['원금또는이자지급방법'] = df['원금또는이자지급방법'].fillna('') + df['이자의지급방법'].fillna('')
# df['원금또는이자지급방법'] = df['원금또는이자지급방법'].fillna('') + df['이자지급방법'].fillna('')
# df.drop(['이자의지급방법', '이자지급방법'], axis=1, inplace=True)

# df['기본이자율'] = df['기본이자율'].fillna('') + df['이자율'].fillna('')
# df.drop(['이자율'], axis=1, inplace=True)

# df['예금자보호'] = df['예금자보호'].fillna('') + df['예금자보호여부'].fillna('')
# df.drop(['예금자보호여부'], axis=1, inplace=True)

# df['판매기간및한도'] = df['판매기간및한도'].fillna('') + df['판매한도'].fillna('')
# df.drop(['판매한도'], axis=1, inplace=True)

# df['우대서비스'] = df['우대서비스'].fillna('') + df['기본우대서비스'].fillna('')
# df['우대서비스'] = df['우대서비스'].fillna('') + df['우대서비스제공'].fillna('')
# df.drop(['기본우대서비스', '우대서비스제공'], axis=1, inplace=True)

# filtered_df = df[[col for col in df.columns if col in ["우대서비스", "기본우대서비스", "우대서비스제공"]]]
# # filtered_df = df[[col for col in df.columns if "우대서비스" in col]]
# rows_with_values = filtered_df[filtered_df.apply(lambda row: row.count() >= 2, axis=1)]
# print(len(rows_with_values))
df[[col for col in df.columns if col in ["예금과목", "상품명"]]]

# for col in df.columns:
#     num = len(df[~df[col].isna()])
#     print(col, num)

,상품명,예금과목
0,쏠편한 정기예금,NaN
1,가계당좌예금,가계당좌예금
2,국고예금,국고예금
3,당좌예금,당좌예금
4,수퍼저축예금,수퍼저축예금
...,...,...
82,패밀리 상생 적금,정기적금
83,신한 마이홈 적금,NaN
84,신한 주거래 드림(DREAM) 적금,NaN
85,신한 첫급여 드림(DREAM) 적금,NaN


In [26]:
path = "../data/은행상품/신한_backup_1.csv"
df.to_csv(path, encoding="utf-8-sig")

In [367]:
#### 개별 테스트
pdf_path = "../data/은행상품/신한은행/20200609000000060012LC000030.pdf"
prod_name, prod_feat = read_pdf_use_fitz(pdf_path)
etc = pdf_to_dataframe(pdf_path)

prod_name_index = len(etc)+1
etc.at[prod_name_index, etc.columns[0]] = "상품명"
etc.at[prod_name_index, etc.columns[1]] = prod_name
prod_feat_index = len(etc)+1
etc.at[prod_feat_index, etc.columns[0]] = "상품특징"
etc.at[prod_feat_index, etc.columns[1]] = prod_feat
    
etc.set_index(etc.columns[0], inplace=True)
etc = etc.T.reset_index(drop=True)

# df = pd.concat([df, etc], axis=0, ignore_index=False)
# df.reset_index(drop=True, inplace=True)

path = "../data/은행상품/신한Error_%s.csv"%os.path.basename(pdf_path).split('.')[0]
etc.to_csv(path, encoding='utf-8-sig')

In [358]:
df = pd.DataFrame()
paths = "../data/은행상품/신한은행/*.pdf"
error_log_path = "./error.txt"
error_f = open(error_log_path, "w", encoding="utf-8-sig")

for i, path in enumerate(tqdm(glob(paths))):
    
    try:
        prod_name, prod_feat = read_pdf_use_fitz(path)
        etc = pdf_to_dataframe(path)
        prod_name_index = len(etc)+1
        etc.at[prod_name_index, etc.columns[0]] = "상품명"
        etc.at[prod_name_index, etc.columns[1]] = prod_name
        prod_feat_index = len(etc)+1
        etc.at[prod_feat_index, etc.columns[0]] = "상품특징"
        etc.at[prod_feat_index, etc.columns[1]] = prod_feat
        etc.set_index(etc.columns[0], inplace=True)
        etc = etc.T.reset_index(drop=True)
        
        df = pd.concat([df, etc], axis=0)
        df.reset_index(drop=True, inplace=True)
        
    except Exception as e:
        error_f.write(os.path.basename(path)+"\n")
        traceback.print_exc(file=error_f)  # traceback를 파일에 직접 기록
        pass
error_f.close()

# 변경하고자 하는 특정 컬럼의 이름 설정
target_column = ["상품명", "상품특징"]

# 특정 컬럼의 위치를 변경
columns_except_target = [col for col in df.columns if col not in target_column]
target_column.extend(columns_except_target)
df = df[target_column]

path = "../data/은행상품/신한.csv"
df.to_csv(path, encoding="utf-8-sig")

100%|██████████| 87/87 [01:58<00:00,  1.36s/it]
